In [85]:
# import torch
# from bilstm_crf import BiLSTMCRF
from torch import optim
from utils import *
from tqdm import tqdm
from sklearn.preprocessing import MultiLabelBinarizer


# import evaluation metrics 
from sklearn.metrics import f1_score, precision_score, recall_score, accuracy_score
from sklearn.model_selection import train_test_split

torch.manual_seed(1)

In [86]:
import torch
import torch.nn as nn
from TorchCRF import CRF

class BiLSTMCRF(nn.Module):
    def __init__(self, vocab_size, word_embedding_dim, hidden_dim, output_dim):
        
        super(BiLSTMCRF, self).__init__()
        # hyperparameters
        self.word_embedding_dim = word_embedding_dim
        self.hidden_dim = hidden_dim
        self.vocab_size = vocab_size
        self.output_dim = output_dim

        # model layers
        self.word_embedding = nn.Embedding(vocab_size, word_embedding_dim)
        self.lstm = nn.LSTM(word_embedding_dim, hidden_dim // 2, bidirectional=True, dropout=0.2)
        self.hidden_to_tag = nn.Linear(hidden_dim, output_dim)
        self.crf = CRF(output_dim)

    def __init_hidden(self):
        return (torch.randn(2, 1, self.hidden_dim // 2), torch.randn(2, 1, self.hidden_dim // 2)) # initialize hidden state

    def __get_lstm_features(self, sentence):
        self.hidden = self.__init_hidden()
        word_embeddings = self.word_embedding(sentence).view(len(sentence), 1, -1)
        lstm_out, self.hidden = self.lstm(word_embeddings, self.hidden)
        lstm_features = self.hidden_to_tag(lstm_out)
        return lstm_features

    def neg_log_likelihood(self, sentence, tags):
        features = self.__get_lstm_features(sentence) # emissions from the lstm layer
        tags = tags.view(-1, 1)
        loss = -self.crf(features, tags)
        return loss

    def forward(self, sentence):
        lstm_features = self.__get_lstm_features(sentence) # emissions from the lstm layer
        tag_sequence = self.crf.decode(lstm_features)
        return tag_sequence
    

In [87]:
def read_dataset():
    data = pd.read_csv('../ner_dataset/ner_dataset.csv', encoding='latin1')

    # remove white spaces from column names
    data.columns = data.columns.str.strip()

    print(data.columns)
    # Group by 'Sentence #' and aggregate
    grouped_data = data.groupby('Sentence #').agg({
        'Word': lambda x: ''.join(x),  # Join words into a single sentence
        'Tag': lambda x: list(x.str.strip()),       # Collect tags into a list
        'Intent': lambda x: list(x.str.strip().str.replace('_', ' '))     # Collect intents into a list
    }).reset_index()  # Reset index to make 'Sentence #' a regular column

    return data, grouped_data


def prepare_data(dataframe):
    dataset = []
    for _, row in dataframe.iterrows():
        sentence = row['Word'][1:]
        tags = row['Tag']
        intents = row['Intent']
        dataset.append((sentence, tags, intents[0]))

    return dataset

In [88]:
# Format data
data, goruped_data = read_dataset()
print(data["Intent"].unique())

goruped_data.head()

Index(['Sentence #', 'Word', 'Tag', 'Intent'], dtype='object')
[' variable declaration' ' function declaration' ' class declaration'
 ' assignment operation' ' array operation' ' bitwise operation'
 ' mathematical operation' ' membership operation' ' casting'
 ' io operation' ' assertion' ' libraries' ' file system' ' ide operation'
 ' comment' ' conditional operation' ' iterative operation']


,Sentence #,Word,Tag,Intent
0,0,make start time as double and initialize 0.00...,"[O, B-VAR, I-VAR, O, B-TYPE, O, O, B-VAL]","[variable declaration, variable declaration, v..."
1,1,declare min value as integer and value 131313,"[O, B-VAR, I-VAR, O, B-TYPE, O, O, B-VAL]","[variable declaration, variable declaration, v..."
2,2,define settings as boolean and value false,"[O, B-VAR, O, B-TYPE, O, O, B-VAL]","[variable declaration, variable declaration, v..."
3,3,define y as integer and assign to 12345,"[O, B-VAR, O, B-TYPE, O, O, O, B-VAL]","[variable declaration, variable declaration, v..."
4,4,initialize k as string and initialize it with...,"[O, B-VAR, O, B-TYPE, O, O, O, O, B-VAL, I-VAL]","[variable declaration, variable declaration, v..."


In [89]:
training_data = prepare_data(goruped_data)

sentences = [sentence for sentence, _, _ in training_data]
tags = [tag for _, tag, _ in training_data]
intents = set([intent for _, _, intent in training_data])

intents

{'array operation',
 'assertion',
 'assignment operation',
 'bitwise operation',
 'casting',
 'class declaration',
 'comment',
 'conditional operation',
 'file system',
 'function declaration',
 'ide operation',
 'io operation',
 'iterative operation',
 'libraries',
 'mathematical operation',
 'membership operation',
 'variable declaration'}

In [90]:
separate_data = {} # a dictionary to store the data for each intent

# separate training data by intent
for intent in intents:
    separate_data[intent] = [(sentence, tag) for sentence, tag, _intent in training_data if _intent == intent]

for intent in separate_data:
    print(f'Intent: {intent}')
    print(f'Number of sentences: {len(separate_data[intent])}')
    print(f'Example: {separate_data[intent][0]}')
    print('\n')

Intent: conditional operation
Number of sentences: 160
Example: ('whether j less than -2020202', ['O', 'B-LHS', 'B-CONDITION', 'I-CONDITION', 'B-RHS'])


Intent: mathematical operation
Number of sentences: 370
Example: ('divide order queue and j', ['B-OPERATOR', 'B-OPERAND', 'I-OPERAND', 'O', 'B-OPERAND'])


Intent: casting
Number of sentences: 78
Example: ('make budget allocation to int', ['O', 'B-VAR', 'I-VAR', 'O', 'B-CAST_TYPE'])


Intent: comment
Number of sentences: 50
Example: ('comment a This is a date comment', ['I-COMMENT', 'I-COMMENT', 'B-COMMENT', 'I-COMMENT', 'O', 'I-COMMENT', 'O'])


Intent: function declaration
Number of sentences: 112
Example: ('write a procedure named rayleigh quotient iteration algorithm', ['O', 'O', 'O', 'O', 'B-FUNC', 'I-FUNC', 'I-FUNC', 'I-FUNC'])


Intent: ide operation
Number of sentences: 230
Example: ('launch terminal', ['B-ACTION', 'B-TYPE'])


Intent: membership operation
Number of sentences: 114
Example: ('check whether item array is in set 

In [109]:
separate_word_to_index = {}
separate_tag_to_index = {}
separate_index_to_tag = {}

for intent in separate_data:
    words = [word for sentence, _ in separate_data[intent] for word in sentence.split()]
    tags = [tag for _, tag in separate_data[intent] for tag in tag]

    word_to_index = {word: i + 1 for i, word in enumerate(set(words))}
    tag_to_index = {tag: i + 1 for i, tag in enumerate(set(tags))}

    word_to_index['<UNK>'] = 0
    tag_to_index['<UNK>'] = 0

    separate_word_to_index[intent] = word_to_index
    separate_tag_to_index[intent] = tag_to_index

separate_index_to_tag = {intent: {i: tag for tag, i in separate_tag_to_index[intent].items()} for intent in separate_tag_to_index}

for intent in separate_data:
    print(f'Intent: {intent}')
    print(f'Number of unique words: {len(separate_word_to_index[intent])}')
    print(f'Number of unique tags: {len(separate_tag_to_index[intent])}')
    print('\n')

Intent: conditional operation
Number of unique words: 283
Number of unique tags: 9


Intent: mathematical operation
Number of unique words: 285
Number of unique tags: 7


Intent: casting
Number of unique words: 112
Number of unique tags: 5


Intent: comment
Number of unique words: 47
Number of unique tags: 4


Intent: function declaration
Number of unique words: 173
Number of unique tags: 7


Intent: ide operation
Number of unique words: 115
Number of unique tags: 7


Intent: membership operation
Number of unique words: 249
Number of unique tags: 8


Intent: class declaration
Number of unique words: 58
Number of unique tags: 4


Intent: assignment operation
Number of unique words: 178
Number of unique tags: 6


Intent: assertion
Number of unique words: 117
Number of unique tags: 8


Intent: variable declaration
Number of unique words: 298
Number of unique tags: 7


Intent: iterative operation
Number of unique words: 162
Number of unique tags: 12


Intent: libraries
Number of unique wor

In [92]:
with open('../ner_dataset/intent_to_tags.json', 'r') as f:
    intent_to_tags = json.load(f)

final_intent_to_tags = {}

for intent in intent_to_tags.keys():
    final_intent_to_tags[intent] = []
    for tag in intent_to_tags[intent]:
        if tag == 'O':
            final_intent_to_tags[intent].append(tag)
        else:
            final_intent_to_tags[intent].append('B-' + tag)
            final_intent_to_tags[intent].append('I-' + tag)

        
final_intent_to_tags

{'variable declaration': ['O',
  'B-VAR',
  'I-VAR',
  'B-VAL',
  'I-VAL',
  'B-TYPE',
  'I-TYPE'],
 'function declaration': ['O',
  'B-FUNC',
  'I-FUNC',
  'B-PARAM',
  'I-PARAM',
  'B-RET_TYPE',
  'I-RET_TYPE'],
 'class declaration': ['O', 'B-CLASS_NAME', 'I-CLASS_NAME'],
 'assignment operation': ['O', 'B-LHS', 'I-LHS', 'B-RHS', 'I-RHS'],
 'conditional operation': ['O',
  'B-LHS',
  'I-LHS',
  'B-RHS',
  'I-RHS',
  'B-CONDITION',
  'I-CONDITION',
  'B-LOG',
  'I-LOG'],
 'iterative operation': ['O',
  'B-LOOP',
  'I-LOOP',
  'B-START',
  'I-START',
  'B-END',
  'I-END',
  'B-LHS',
  'I-LHS',
  'B-RHS',
  'I-RHS',
  'B-CONDITION',
  'I-CONDITION',
  'B-STEP',
  'I-STEP'],
 'array operation': ['O',
  'B-ARRAY',
  'I-ARRAY',
  'B-OPERATION',
  'I-OPERATION',
  'B-ELEMENT',
  'I-ELEMENT'],
 'bitwise operation': ['O',
  'B-OPERATOR',
  'I-OPERATOR',
  'B-OPERAND',
  'I-OPERAND'],
 'mathematical operation': ['O',
  'B-OPERAND',
  'I-OPERAND',
  'B-OPERATOR',
  'I-OPERATOR',
  'B-VAR',
  'I-

In [93]:
# training data for each intent
# training_data = {intent: [x_train, y_train, x_test, y_test]}
training_data = {} 
for intent in separate_data:
    sentences = [sentence for sentence, _ in separate_data[intent]]
    tags = [tag for _, tag in separate_data[intent]]
    x_train, x_test, y_train, y_test = train_test_split(sentences, tags, test_size=0.1, random_state=42)
    training_data[intent] = [x_train, y_train, x_test, y_test]



In [96]:
# models = {
#    intent: [model, optimizer]}
models = {}

for intent in final_intent_to_tags.keys():
    models[intent] = []
    models[intent].append(BiLSTMCRF(vocab_size=len(separate_word_to_index[intent]), word_embedding_dim=100, hidden_dim=128, output_dim=len(separate_tag_to_index[intent])))
    models[intent].append(optim.SGD(models[intent][0].parameters(), lr=0.01, weight_decay=1e-4))

In [98]:
# Check predictions before training
# tarining data {
#     intent: [x_train, y_train, x_test, y_test]
# }
with torch.no_grad():
    for intent in intents:
        print(f'Intent: {intent}')
        precheck_sent = prepare_sequence(training_data[intent][0][0].split(), separate_word_to_index[intent])
        precheck_tag = prepare_sequence(training_data[intent][1][0], separate_tag_to_index[intent])
        print("Actual tags:", precheck_tag)
        print("Predicted tags:", models[intent][0](precheck_sent))


Intent: conditional operation
Actual tags: tensor([3, 5, 2, 3, 6, 7, 3, 8, 1, 4, 3, 3, 3, 6, 3, 3, 3])
Predicted tags: [[0, 8, 7, 0, 4, 7, 7, 7, 0, 4, 8, 7, 7, 7, 7, 7, 7]]
Intent: mathematical operation
Actual tags: tensor([2, 4, 3, 4])
Predicted tags: [[2, 2, 2, 6]]
Intent: casting
Actual tags: tensor([4, 4, 4, 3, 1, 4, 4, 2])
Predicted tags: [[3, 0, 2, 0, 3, 3, 0, 0]]
Intent: comment
Actual tags: tensor([2, 1, 3, 3])
Predicted tags: [[1, 2, 2, 2]]
Intent: function declaration
Actual tags: tensor([3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 1, 5, 3, 3, 3])
Predicted tags: [[4, 4, 0, 3, 0, 2, 3, 6, 4, 2, 6, 6, 1, 6, 1]]
Intent: ide operation
Actual tags: tensor([1])
Predicted tags: [[2]]
Intent: membership operation
Actual tags: tensor([3, 4, 3, 7, 3, 3, 5])
Predicted tags: [[7, 4, 0, 4, 2, 4, 2]]
Intent: class declaration
Actual tags: tensor([2, 3, 2, 2, 2])
Predicted tags: [[2, 1, 2, 1, 2]]
Intent: assignment operation
Actual tags: tensor([3, 1, 2, 4])
Predicted tags: [[5, 2, 3, 5]]
Intent: asser

In [99]:
# training loop
def train(model, optimizer, x_train, y_train,epochs=10):
    for epoch in tqdm(range(epochs)):
        total_loss = 0
        for sentence, tags in zip(x_train, y_train):
            # Step 1. Remember that Pytorch accumulates gradients.
            # We need to clear them out before each instance
            model.zero_grad()
        
            sentence = prepare_sequence(sentence.split(), separate_word_to_index[intent])
            target_tags = prepare_sequence(tags, separate_tag_to_index[intent])

            # Step 3. Run our forward pass.
            loss = model.neg_log_likelihood(sentence, target_tags)

            total_loss += loss.item()

            # Step 4. Compute the loss, gradients, and update the parameters by
            loss.backward()
            optimizer.step()
        
        print(f"Epoch: {epoch}, Loss: {total_loss / len(x_train)}")

In [100]:
for intent in intents:
    x_train, y_train = training_data[intent][0], training_data[intent][1]
    train(models[intent][0], models[intent][1], x_train, y_train, epochs=20)

  5%|▌         | 1/20 [00:03<01:09,  3.63s/it]

Epoch: 0, Loss: 16.38363363676601


 10%|█         | 2/20 [00:05<00:50,  2.82s/it]

Epoch: 1, Loss: 8.70021105143759


 15%|█▌        | 3/20 [00:07<00:41,  2.46s/it]

...

Epoch: 17, Loss: 0.34391419640902815


 95%|█████████▌| 19/20 [00:10<00:00,  1.72it/s]

Epoch: 18, Loss: 0.3542645772298177


100%|██████████| 20/20 [00:10<00:00,  1.84it/s]

Epoch: 19, Loss: 0.3301591982786683


In [101]:
# Check predictions before training
# tarining data {
#     intent: [x_train, y_train, x_test, y_test]
# }
with torch.no_grad():
    for intent in intents:
        print(f'Intent: {intent}')
        precheck_sent = prepare_sequence(training_data[intent][0][0].split(), separate_word_to_index[intent])
        precheck_tag = prepare_sequence(training_data[intent][1][0], separate_tag_to_index[intent])
        print("Actual tags:", precheck_tag)
        print("Predicted tags:", models[intent][0](precheck_sent))


Intent: conditional operation
Actual tags: tensor([3, 5, 2, 3, 6, 7, 3, 8, 1, 4, 3, 3, 3, 6, 3, 3, 3])
Predicted tags: [[3, 5, 2, 3, 6, 7, 3, 8, 1, 4, 3, 3, 3, 6, 3, 3, 3]]
Intent: mathematical operation
Actual tags: tensor([2, 4, 3, 4])
Predicted tags: [[2, 4, 3, 4]]
Intent: casting
Actual tags: tensor([4, 4, 4, 3, 1, 4, 4, 2])
Predicted tags: [[4, 4, 4, 3, 1, 4, 4, 2]]
Intent: comment
Actual tags: tensor([2, 1, 3, 3])
Predicted tags: [[2, 1, 3, 3]]
Intent: function declaration
Actual tags: tensor([3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 1, 5, 3, 3, 3])
Predicted tags: [[3, 3, 3, 3, 3, 3, 4, 3, 3, 3, 1, 2, 3, 3, 3]]
Intent: ide operation
Actual tags: tensor([1])
Predicted tags: [[1]]
Intent: membership operation
Actual tags: tensor([3, 4, 3, 7, 3, 3, 5])
Predicted tags: [[3, 4, 3, 7, 3, 3, 5]]
Intent: class declaration
Actual tags: tensor([2, 3, 2, 2, 2])
Predicted tags: [[2, 3, 2, 2, 2]]
Intent: assignment operation
Actual tags: tensor([3, 1, 2, 4])
Predicted tags: [[3, 1, 2, 4]]
Intent: asser

In [103]:
def predict(model, test_data):
    model.eval()
    predictions = [] # a dictionary to store the predictions for each intent
    with torch.no_grad():
        for sentence in test_data:
            precheck_sent = prepare_sequence(sentence.split(), separate_word_to_index[intent])
            predictions.append(model(precheck_sent)[0])
    return predictions

In [104]:
# evaluation 
def evaluate(y_true, y_pred):
    mlb = MultiLabelBinarizer()
    y_true_binarized = mlb.fit_transform(y_true)
    y_pred_binarized = mlb.transform(y_pred)
    
    f1 = f1_score(y_true_binarized, y_pred_binarized, average='weighted')
    precision = precision_score(y_true_binarized, y_pred_binarized, average='weighted')
    recall = recall_score(y_true_binarized, y_pred_binarized, average='weighted')
    accuracy = accuracy_score(y_true_binarized, y_pred_binarized)
    
    return f1, precision, recall, accuracy

In [111]:
for intent in intents:
    print(f'Intent: {intent}')
    x_test, y_test = training_data[intent][2], training_data[intent][3]
    predictions = predict(models[intent][0], x_test)

    mapped_predictions = [[separate_index_to_tag[intent][tag] for tag in tags] for tags in predictions]

    f1, precision, recall, accuracy = evaluate(y_test, mapped_predictions)
    print("F1 Score:", f1)
    print("Precision:", precision)
    print("Recall:", recall)
    print("Accuracy:", accuracy)
    print("----------------------------------------------------------------------")

Intent: conditional operation
F1 Score: 0.9656853924365445
Precision: 0.9826839826839827
Recall: 0.9523809523809523
Accuracy: 0.6875
----------------------------------------------------------------------
Intent: mathematical operation
F1 Score: 0.9841082929170324
Precision: 0.9876169510181618
Recall: 0.9810126582278481
Accuracy: 0.918918918918919
----------------------------------------------------------------------
Intent: casting
F1 Score: 0.9808429118773946
Precision: 1.0
Recall: 0.9655172413793104
Accuracy: 0.875
----------------------------------------------------------------------
Intent: comment
F1 Score: 1.0
Precision: 1.0
Recall: 1.0
Accuracy: 1.0
----------------------------------------------------------------------
Intent: function declaration
F1 Score: 0.9891304347826088
Precision: 1.0
Recall: 0.9791666666666666
Accuracy: 0.9166666666666666
----------------------------------------------------------------------
Intent: ide operation
F1 Score: 0.9913194444444444
Precision: 1.

c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning: unknown class(es) ['B-VAR', 'I-VAR'] will be ignored
  warnings.warn(


In [113]:
# Check predictions before training
test_example_1 = ("create a variable called ay khara and set it to yasmine ghanem", ["O", "O", "O", "O", "B-VAR", "I-VAR", "O", "O", "O", "B-VAL", "I-VAL"], "variable declaration")
test_example_2 = ("cast x to an int", ["O", "B-VAR", "O", "O", "B-CAST_TYPE"], "casting")
test_example_3 = ("import the pandas library", ["O", "O", "B-LIB_NAME", "O"], "libraries")
test_example_4 = ('write a new comment multiply the emissions and transitions', ['O', 'O', 'O', 'B-COMMENT', 'I-COMMENT', 'I-COMMENT', 'I-COMMENT', 'I-COMMENT' ], 'comment')
test_example_5 = ('define a new class person', ['O', 'O', 'O', 'O', 'B-CLASS_NAME'], 'class declaration')
test_example_6 = ('create a new function named add that takes parameters x and y', ['O', 'O', 'O', 'O', 'O', 'B-FUNC', 'O', 'O', 'O', 'B-PARAM', 'O', 'B-PARAM'], 'function declaration')
test_example_7 = ('create a new function called add with parameters num1 and num2', ['O', 'O', 'O', 'O', 'O', 'B-FUNC', 'O', 'O', 'O', 'B-PARAM', 'O', 'B-PARAM'], 'function declaration')

test_data = [test_example_1, test_example_2, test_example_3, test_example_4, test_example_5, test_example_6, test_example_7]

for index, example in enumerate(test_data):
    print("Sentence:", example[0])
    print("Intent:", example[2])

    predictions = predict(models[example[2]][0], [example[0]])

    mapped_predictions = [[separate_index_to_tag[example[2]][tag] for tag in tags] for tags in predictions]

    print("Actual tags:", example[1])
    print("Predicted tags:", mapped_predictions[0])

    evaluated = evaluate([example[1]], mapped_predictions)

    print("----------------------------------------------------------------------------------------------------")

# predictions = predict(test_model, test_data)
# labels = []
# predicted_tags = []

# for index, prediction in enumerate(predictions):
#     print("Test Example:", index)
#     mapped_tags = [all_tags[tag] for tag in prediction]
#     predicted_tags.append(mapped_tags)
#     labels.append(test_data[index][1])
#     print("Actual tags:", test_data[index][1])
#     print("Predicted tags:", mapped_tags)

Sentence: create a variable called ay khara and set it to yasmine ghanem
Intent: variable declaration
Actual tags: ['O', 'O', 'O', 'O', 'B-VAR', 'I-VAR', 'O', 'O', 'O', 'B-VAL', 'I-VAL']
Predicted tags: ['O', 'B-VAR', 'I-VAR', 'B-VAR', 'I-VAR', 'O', 'B-VAR', 'I-VAR', 'O', 'B-VAR', 'I-VAR', 'I-VAL']
----------------------------------------------------------------------------------------------------
Sentence: cast x to an int
Intent: casting
Actual tags: ['O', 'B-VAR', 'O', 'O', 'B-CAST_TYPE']
Predicted tags: ['O', 'B-VAR', 'I-VAR', 'O', 'B-CAST_TYPE']
----------------------------------------------------------------------------------------------------
Sentence: import the pandas library
Intent: libraries
Actual tags: ['O', 'O', 'B-LIB_NAME', 'O']
Predicted tags: ['O', 'B-LIB_NAME', 'O', 'B-LIB_NAME']
----------------------------------------------------------------------------------------------------
Sentence: write a new comment multiply the emissions and transitions
Intent: comment


c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\yazmi\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\preprocessing\_label.py:895: UserWarning: unknown class(es) ['I-VAR'] will be ignored
  warnings.warn(


IndexError: index out of range in self